In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col, avg, concat, lit, from_json
from pyspark.sql.types import StructType, StructField, LongType, StringType, IntegerType, BooleanType
from time import sleep

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Lab9_Ex3")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

dataSchema = StructType([
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Price", IntegerType(), True),
    StructField("Lot_size", StringType(), True),
    StructField("Living_space_size", StringType(), True),
    StructField("Build_year", StringType(), True),
    StructField("Build_type", StringType(), True),
    StructField("House_type", StringType(), True),
    StructField("Roof", StringType(), True),
    StructField("Rooms", StringType(), True),
    StructField("Toilet", StringType(), True),
    StructField("Floors", StringType(), True),
    StructField("Energy_label", StringType(), True),
    StructField("Position", StringType(), True),
    StructField("Garden", StringType(), True),
    StructField("Estimated_neighbourhood_price_per", StringType(), True),
    StructField("Availability", BooleanType(), True),
])


# Read the whole dataset as a batch
kafkaStream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9093") \
    .option("failOnDataLoss", "false") \
    .option("subscribe", "mock") \
    .option("startingOffsets", "latest") \
    .load()

df = kafkaStream.selectExpr("CAST(value AS STRING)")

df1 = df.select(from_json(df.value, dataSchema.simpleString()))

df1.printSchema()

sdf = df1.select(col("from_json(value).*"))

sdf.printSchema()

avgpricedf = sdf.groupBy( "Address", "City") \
    .agg(avg("Price").alias("value"))

resultdf = avgpricedf.select(concat(col("Address"), lit(" "), col("City")).alias("key"), col("value"))

resultdf.printSchema()

query = resultdf \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9093") \
    .option("checkpointLocation", "/home/jovyan/checkpoint/gamescore") \
    .option("topic", "mock_2") \
    .outputMode("complete") \
    .start()
try:
    query.awaitTermination()
except KeyboardInterrupt:
    query.stop()
    # Stop the spark context
    spark.stop()
    print("Stoped the streaming query and the spark context")

In [ ]:
spark.stop()